In [1]:
import tensorflow as tf
import torch

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv("~/desktop/ml/new.csv", encoding='ISO-8859-1', low_memory=False)
data.head()

,url,id,Lng,Lat,Cid,tradeTime,DOM,followers,totalPrice,price,...,buildingType,constructionTime,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage
0,https://bj.lianjia.com/chengjiao/101084782030....,101084782030,116.475489,40.019520,1111027376244,2016-08-09,1464.0,106,415.0,31680,...,1.0,2005,3,6,0.217,1.0,0.0,1.0,7,56021.0
1,https://bj.lianjia.com/chengjiao/101086012217....,101086012217,116.453917,39.881534,1111027381879,2016-07-28,903.0,126,575.0,43436,...,1.0,2004,4,6,0.667,1.0,1.0,0.0,7,71539.0
2,https://bj.lianjia.com/chengjiao/101086041636....,101086041636,116.561978,39.877145,1111040862969,2016-12-11,1271.0,48,1030.0,52021,...,4.0,2005,3,6,0.500,1.0,0.0,0.0,7,48160.0
3,https://bj.lianjia.com/chengjiao/101086406841....,101086406841,116.438010,40.076114,1111043185817,2016-09-30,965.0,138,297.5,22202,...,1.0,2008,1,6,0.273,1.0,0.0,0.0,6,51238.0
4,https://bj.lianjia.com/chengjiao/101086920653....,101086920653,116.428392,39.886229,1111027381174,2016-08-28,927.0,286,392.0,48396,...,4.0,1960,2,2,0.333,0.0,1.0,1.0,1,62588.0


In [4]:
import plotly.express as px

In [5]:
#px.scatter_3d(data, x = "square", y = "tradeTime", z = "totalPrice")


In [6]:
from util import load_data, add_theta_0
from sklearn.preprocessing import StandardScaler


In [7]:
def load(file):
    x, y = load_data(file, np.array(['square', 'tradeTime']), label='totalPrice', intialize_theta_0=False, type="str")
    #print(x)
    x[:, 0] = [str[0:4] for str in x[:, 0]]
    m, n = x.shape
    #print(x[:10])
    #print(m)
    x = x.astype(float)
    y = y.astype(float)
    
    

    
    #x = scaler.fit_transform(x)
    
    #y = y.reshape(-1, 1)
    #y = scaler.fit_transform(y)
    
    
    return x, y

In [126]:
x_train, y_train = load("new_1.csv")
y_train = y_train.reshape(-1, 1)

scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_train = add_theta_0(x_train)
x_train



array([[ 1.        ,  0.73707906,  1.32799619],
       [ 1.        ,  0.73707906, -0.27457176],
       [ 1.        ,  0.73707906,  4.17850523],
       ...,
       [ 1.        ,  0.73707906, -1.35707429],
       [ 1.        ,  0.73707906, -1.29769993],
       [ 1.        ,  0.73707906,  0.51267828]])

In [156]:
X = tf.constant(x_train,tf.float32,name="X")
Y = tf.constant(y_train,tf.float32,name="Y")
import time
#Using Normal Equation
def normal_1():
    start = time.perf_counter()
    X_T = tf.transpose(X)
    temp = tf.linalg.inv(tf.matmul(X_T,X))
    theta = tf.matmul(tf.matmul(temp,X_T),Y)
    #print(theta)
    elapsed = time.perf_counter() - start
    return elapsed

def normal_2():
    start = time.perf_counter()
    theta = tf.matmul(tf.matmul(tf.linalg.inv(tf.matmul(tf.transpose(X),X)),tf.transpose(X)),Y)
    #print(theta)
    elapsed = time.perf_counter() - start
    return elapsed

In [157]:
sum1 = 0
sum2 = 0
for i in range(1000):
    sum1 += normal_1()
    sum2 += normal_2()

print("sum1", sum1 / 1000)
print("sum2", sum2 / 1000)

sum1 0.015269809850360616
sum2 0.022575512271068875


In [158]:
from sklearn.preprocessing import StandardScaler

n_epoch = 100000
learning_rate = 0.1
m, n = x_train.shape

y_train = y_train.reshape(-1, 1)
X = tf.constant(x_train, dtype = tf.float32, name = "Scaled Features")

theta = tf.Variable((tf.random.uniform([3, 1], -1.0, 1.0)), name = "theta")
#theta = tf.Variable(([[0.], [0.], [0.]]), name = "theta")

X

<tf.Tensor: shape=(95655, 3), dtype=float32, numpy=
array([[ 1.        ,  0.7370791 ,  1.3279961 ],
       [ 1.        ,  0.7370791 , -0.27457175],
       [ 1.        ,  0.7370791 ,  4.1785054 ],
       ...,
       [ 1.        ,  0.7370791 , -1.3570743 ],
       [ 1.        ,  0.7370791 , -1.2976999 ],
       [ 1.        ,  0.7370791 ,  0.51267827]], dtype=float32)>

In [159]:
import time
start = time.perf_counter()

for epoch in range(n_epoch):
   y_pred = tf.matmul(X, theta, name = "predictions")

   error = y_pred - y_train
   
   mse = tf.reduce_mean(tf.square(error), name = "mse")
   #gradients = 1/m * tf.matmul(tf.transpose(X), error)
   theta_old = theta
   #theta = theta - learning_rate * gradients
   theta = theta + learning_rate /  (epoch / 500 + 1) * tf.transpose(X) @ (y_train - X @ theta) / m
  
   if (np.linalg.norm(theta_old - theta) < 0.00001):
        print(theta)
        print("Epoch: ", epoch, "MSE: ", mse)
        print(epoch)
        break
   if epoch % 100 == 0:
        #print("1", learning_rate * gradients)
        #print("2", learning_rate * tf.transpose(X) @ (y_train - X @ theta) / m)
        print("Epoch: ", epoch, "MSE: ", mse)


elapsed = time.perf_counter() - start
print('Elapsed %.3f seconds.' % elapsed)

Epoch:  0 MSE:  tf.Tensor(173563.45, shape=(), dtype=float32)
Epoch:  100 MSE:  tf.Tensor(26707.95, shape=(), dtype=float32)
Epoch:  200 MSE:  tf.Tensor(26695.125, shape=(), dtype=float32)
Epoch:  300 MSE:  tf.Tensor(26695.123, shape=(), dtype=float32)
tf.Tensor(
[[349.0764  ]
 [ 91.484474]
 [129.80608 ]], shape=(3, 1), dtype=float32)
Epoch:  343 MSE:  tf.Tensor(26695.123, shape=(), dtype=float32)
343
Elapsed 1.424 seconds.


In [ ]:
dd

In [36]:
x_test, y_test = load("new_2.csv")

x_test = scaler.transform(x_test)
x_test = add_theta_0(x_test)

y_test = y_test.reshape(-1, 1)

In [37]:
a = x_test @ theta
np.mean((a - y_test)**2)

27550.87

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import LambdaCallback

model = Sequential()
model.add(Dense(1, input_dim = 2))

print_weights = LambdaCallback(on_epoch_end=lambda batch, logs: print(model.layers[0].get_weights(), model.layers[0].get_weights()[1]))

model.compile(loss = 'mse')
model.fit(x = x_train, y = y_train, epochs = 20, verbose = 1,callbacks = [print_weights])